Task 1: Third-order letter approximation model

In [46]:
import random

First cell defines needed perameters such as links to files.
A dictionary to add each text to.
And Strings to define preamble and postable markers for removal.

In [47]:
files = [
    '/workspaces/emergingtechnologies/data/frankenstein.txt',
    '/workspaces/emergingtechnologies/data/bookplates.txt',
    '/workspaces/emergingtechnologies/data/spyglassmoutain.txt',
    '/workspaces/emergingtechnologies/data/theoldhouse.txt',
    '/workspaces/emergingtechnologies/data/windofdestiny.txt',
]

Comparisons = '/workspaces/emergingtechnologies/data/words.txt'

texts = {}

#preamble and postamble markers
start_marker = " ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"

startseed = "TH"
length = 10000

We then need to read each file, rather then doing this individually I've created a loop which adds each text to a dictionary.I've done it this way so I dont have to deal with indivdual texts and can simply loop over the dictionary when required to retrieve each text.

After this I used the .find feature within the same loop, to find the markers that I've predefined and extract all text between these markers using .strip, this removes the preamble and postamble. I then add this cleaned text to the dictionary with filename as the key as this allows me to print each text separatly to check each is cleaned and for future use.

In [48]:
#read each file and remove preamble and postamble
for file_path in files:
    with open(file_path, 'r') as f:
        text = f.read()

        #find the start and end markers
        start = text.find(start_marker)
        end = text.find(end_marker)

        #skip the start marker
        start += len(start_marker)
        #extract all text between the markers
        text = text[start:end].strip()

        #store the cleaned text in the dictionary with filename as the key
        filename = file_path.split('/')[-1]  # Get filename from path
        texts[filename] = text

I then create a new loop that iterates over each key-value pair in the texts dictionary. I had to add a .replace method to this to remove '/n' at it was showing up a number of times. I then define the charecters allowed in text to be fed to the model. Then changing my text to uppercase I clean the the text by removing any charecters not in the defined char set, before adding the cleaned text back to its key in the dictionary.

In [49]:
for filename, text in texts.items():
    #remove all newlines
    text = text.replace('\n', ' ')    
    text = text.upper()
    chars = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ. ')
    #remove all characters that are not in the set of allowed characters
    cleaned = ''.join(c for c in text if c in chars)
    #store the cleaned text back in the dictionary
    texts[filename] = cleaned

#print(texts['theoldhouse.txt'])

I then created a trigram model. Using the dictionary model and the same key value loop as before, I created a loop that iterates over the file provided in each loop, each trigram is extracted from the text. A if statement was required to remove empty values that were missed in the cleaning process. These trigrams are then added to the dictionary as a key with each occurance being counted and added to the value

In [50]:
model = {}

for filename, cleaned in texts.items():
  #iterate over the text and count the occurrences of each trigram
  for i in range(len(cleaned) - 2):
    #extract the trigram
    trigram = cleaned[i:i+3]
    if trigram != '' and trigram != '   ':
        #store the trigram in the model
      model[trigram] = model.get(trigram, 0) + 1

This is just a sorted example of the model.

In [51]:
#sorted(model.items(), key=lambda x: x[1], reverse=True)

Task 2: Third-order letter approximation generation

I first created a trigram dictionary to store the bigrams with next char counts, I then loop over the existing trigram and extract each bigram from it. I then collect the next charecter from each bigram and store it as next char, I then add the bigram as the key with the next char as the value and its count.

In [52]:
#trigram look up dictionary for model
trigram_dict = {}

#loop through the model and its count in the model
for trigram, count in model.items():
    #exsure trigram has 3 characters
    if len(trigram) == 3:
        #extract the bigram
        bigram = trigram[:2]
        #extract the next character
        next_char = trigram[2]
        #check bigram is not already accounted for
        if bigram not in trigram_dict:
            #add the bigram to the dictionary
            trigram_dict[bigram] = {}
        #add the next character to the dictionary
        trigram_dict[bigram][next_char] = count

# Manually format and print the trigram dictionary
#for bigram, next_chars in trigram_dict.items():
    #print(f"Bigram '{bigram}':")
    #for next_char, count in next_chars.items():
        #print(f"  '{next_char}': {count}")
    #print()  # Blank line for readability between bigrams

I then need to test to see if I can generate text, I add the startseed which is TH to the generated text to exsure it always starts with TH. I then add this generated text to the bigram to look up the next chars and weights, I then randomly select a next charecter using the weights as probabilities I then add this next charecter to the generated text print this shows a trigram with the last letter randomly selected. Now that I have shown I can generate a next charecter I'll loop this process to repeat until 10,000 charecters are generated.

In [53]:
#initialize the generated text with the start seed
generatedtext = startseed

#get the last two characters in the generated text
bigram = generatedtext[-2:]

#get possible next characters and their counts
next_chars = list(trigram_dict[bigram].keys())
weights = list(trigram_dict[bigram].values())
    
#randomly select the next character based on weights
next_char = random.choices(next_chars, weights=weights, k=1)[0]
    
#next character to the generated text
generatedtext += next_char


#print the generated text with one additional character
print(generatedtext)

THE


Here a essentially do the exact same as in the last cell but add it to a loop with the specified lenght of 10,000 charecters, this means the model will keep reading the last 2 letters in generated text and generating a third letter based on probability from the weights untill the loop is broken.

In [54]:
#loop to generate characters until the length is reached
for _ in range(length):
    
    #get the last two characters in the generated text to form a bigram
    bigram = generatedtext[-2:]
    #get possible character and weights from trigram_dict
    next_chars = list(trigram_dict[bigram].keys()) #list of possible next characters for the bigram
    weights = list(trigram_dict[bigram].values())  #counts used as weights
    #select the next character based on the weights as probabilities
    next_char = random.choices(next_chars, weights=weights, k=1)[0]
    
    #chosen character to the generated text
    generatedtext += next_char

#print(generatedtext)


Task 3: Analyze your model

In [55]:
ComparisonSet = set()

#read the comparison file
with open(Comparisons, 'r') as f:
    for line in f:
        ComparisonSet.add(line.strip())

print(ComparisonSet)

{'WOKE', 'MERITORIOUS', 'ENGLISH', 'LYMAN', 'CONSIDERED', 'POCKETBOOKS', 'GIRAFFE', 'FORMULATED', 'DECIDES', 'GIVE', 'PATROLLING', 'GAINFUL', 'BIZET', 'QUESTING', 'FILLER', 'ICELANDIC', 'BADGER', 'PEAKED', 'PREOCCUPIED', 'PHONE', 'BOWES', 'MUNSEY', 'BRANDENBURG', 'AGAR', 'CRACKLED', 'INKER', 'PATRICIANS', 'ETHEREALLY', 'SPARSE', 'HELLENIZATION', 'DESCRY', 'SEQUENTIALIZED', 'SHALLOW', 'STARVES', 'CASTLE', 'ESCAPEES', 'AUERBACH', 'DAWNING', 'BRONCHUS', 'TRANSLATABILITY', 'MEGAWATT', 'CASEMENT', 'CHIEF', 'OAKLEY', 'SCHOOLS', 'SCHOOLROOM', 'SEPARABLE', 'CRABS', 'COUNTERSUNK', 'COMPLEXION', 'FIANCE', 'TAUT', 'KENOSHA', 'LATINIZING', 'JUXTAPOSED', 'LEVELED', 'INADVISABLE', 'ABUSING', 'SUMPTUOUS', 'UNANALYZABLE', 'ROTATIONAL', 'TOUCHING', 'JUAN', 'MINISTRY', 'CLEAREST', 'UNREASONABLENESS', 'AZERBAIJAN', 'CHAP', 'WOLFE', 'ISADORE', 'HEELED', 'HOOKUPS', 'BLENHEIM', 'PULSING', 'TRAINEE', 'DUTCHESS', 'TRAINER', 'ALTERNATION', 'NATIVES', 'TRESSES', 'DIVERGE', 'AMPLIFIERS', 'INTERACTIVITY', 'TAMELY